<a href="https://colab.research.google.com/github/6k5wyi/DataScience-With-Crawlering/blob/main/%EC%9D%B8%EC%8A%A4%ED%83%80%EA%B7%B8%EB%9E%A8_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인스타그램 크롤링

## 사전작업

### Colab에서 실행시 실행시켜야 할 부분

In [1]:
!pip install selenium 
!apt-get update # apt install을 정확히 실행하기 위해 ubuntu 업데이트 
!apt install chromium-chromedriver 
!cp /usr/lib/chromium-browser/chromedriver /usr/bin 
import sys 
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 
from selenium import webdriver 
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 5.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic I

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


### local에서 실행시 실행시켜야할 부분

In [2]:
# from selenium import webdriver
# installedwebdriver = 'c:/Temp/chromedriver.exe' #웹드라이버 설치 경로 설정
# driver = webdriver.Chrome(installedwebdriver) 

## 인스타그램 접속하기

In [3]:
import time
driver.get('https://www.instagram.com')
time.sleep(2)

## 인스타그램 로그인 하기

In [4]:
import time

email = '인스타아이디''
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '인스타비번'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

IndexError: ignored

## 인스타그램 검색 결과를 URL로 만들어 주는 함수

In [ ]:
def insta_searching(word):
    url = "http://www.instagram.com/explore/tags/" + word
    return url

## 검색 결과 페이지 접속

In [ ]:
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)

## HTML 에서 첫번째 게시글 찾아서 클릭하기

In [ ]:
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

select_first(driver)

## 게시글 정보 가져오기

In [ ]:
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC',content)
    except:
        content = ''
        
    tags = re.findall(r'#[^\s#,\\]+',content) #본문 내용에서 해시태그 가져오기 (정규식 활용)
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #작성일자 정보가져오기
    
    try : # 좋아요 수 가져오기
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except : 
        like = 0
        
    try : # 위치정보 가져오기
        place = soup.select('div.M30cS')[0].text[0]
        place = unicodedata.normalize('NFC',content)
    except : 
        place = ''
    
    data = [content,date,like,place,tags] # 수집된 정보 저장
    return data
    
get_content(driver)    

## 다음 게시글 열기

In [ ]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
move_next(driver)

## 여러 게시글 정보 수집하기

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

driver = webdriver.Chrome('c:/Temp/chromedriver.exe') 

driver.get('https://www.instargram.com')
time.sleep(2)


email = '인스타아이디' 
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '인스타비번' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [ ]:
word = "제주도맛집"   
url = insta_searching(word)


driver.get(url)
time.sleep(3)


select_first(driver)


results = [ ] # 비어있는 변수(results)만들기


# 여러 게시물 수집하기
target = 500      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

## 수집 데이터 저장하기

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
# results_df.to_excel('./insta_crawling_jejudoMatJip.xlsx', index=False)
results_df.to_excel('./Soonchan.xlsx', index=False)

## 중복 데이터 제거

In [ ]:
results_df.drop_duplicates(subset=['content'], inplace = True)
results_df.to_excel('./no_jungbok_insta_crawling_jejudoMatJip.xlsx', index=False)